<a href="https://colab.research.google.com/github/Codeblue-AI/Codeblue-AI/blob/main/ResNet_plain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np

from tqdm.auto import tqdm

In [ ]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
class CustomCIFAR10(Dataset) :
    def __init__(self, root, train=True, transform=None, download=True) :
        self.data = datasets.CIFAR10(root=root, train=train, download=download)
        self.transform = transform

    def __len__(self) :
        return len(self.data)

    def __getitem__(self, idx) :
        image, label = self.data[idx]
        if self.transform :
            image = self.transform(image)
        return image, label

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
batch_size = 64
train_dataset = CustomCIFAR10(root='./data', train=True, transform=transform)
test_dataset = CustomCIFAR10(root='./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

images, labels = next(iter(train_loader))
print("Images batch shape:", images.shape)
print("Labels batch shape:", labels.shape)

In [ ]:
def imshow(img) :
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.figure(figsize=(4,4))
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.axis('off')
    plt.show()

imshow(torchvision.utils.make_grid(images[:8], nrow=4))
print([classes[labels[i]] for i in range(8)])

In [ ]:
class CIFARModel(nn.Module) :
    def __init__(self) :
        super(CIFARModel, self).__init__()

        self.firstlayer = nn.Sequential(
            nn.Conv2d(3,64,3,1,1),
            nn.BatchNorm2d(64)
        )

        self.conv64 = nn.Sequential(
            nn.Conv2d(64, 64, 3, 2, 1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.ReLU()
        )
        self.conv128 = nn.Sequential(
            nn.Conv2d(64, 128, 3, 2, 1),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, 1, 1),
            nn.ReLU()
        )
        self.conv256 = nn.Sequential(
            nn.Conv2d(128, 256, 3, 2, 1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.ReLU()
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(256, 10)

    def forward(self,x) :
        # (3,32,32)
        # print("(3,32,32)", x.shape)
        x = self.firstlayer(x)
        # (64,32,32)
        # print("(64,32,32)", x.shape)
        x = self.conv64(x)
        # (64,16,16)
        # print("(64,16,16)", x.shape)
        x = self.conv128(x)
        # (128,8,8)
        # print("(128,8,8)", x.shape)
        x = self.conv256(x)
        # (256,4,4)
        # print("(256,4,4)", x.shape)
        x = self.avgpool(x)
        x = x.squeeze()
        # (256)
        # print("(256)", x.shape)
        x = self.fc(x)
        # (10)
        # print("(10)", x.shape)

        return x

model = CIFARModel().to(device)
print(model)

In [ ]:
criterion = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epoch_num = 10
running_loss = []
for epoch in tqdm(range(epoch_num)) :
    batch_loss = 0
    for batch_idx, samples in tqdm(enumerate(train_loader)) :
        x_train, y_train = samples
        x_train = x_train.to(device)
        y_train = y_train.to(device)
        y_pred = model(x_train)
        loss = criterion(y_pred, y_train)
        batch_loss += loss.item()

        opt.zero_grad()
        loss.backward()
        opt.step()
    running_loss.append(batch_loss)
    print(f"Train, {epoch}, {batch_loss}")

plt.plot(running_loss)
plt.show()

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad() :
    for images, labels in test_loader :
        images,labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# CIFAR-10 클래스 이름
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

dataiter = iter(test_loader)
images, labels = next(dataiter)

images, labels = images.to(device), labels.to(device)
outputs = model(images)
_, predicted = torch.max(outputs, 1)

imshow(torchvision.utils.make_grid(images[:10].cpu()))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]}' for j in range(10)))
print('Predicted:   ', ' '.join(f'{classes[predicted[j]]}' for j in range(10)))
